In [9]:
ref_text = "ich habe nausea und ein puls von 80 und heute morgen ein aspirin genommen"

In [20]:
hyp_text = "iho habe nausea und ein puls von 80 90 und heute morgen ein arpirin genommen"

In [21]:
import jiwer
# 2. Alignment und Zählung
out = jiwer.process_words(ref_text, hyp_text)

# 3. Auslesen der einzelnen Werte
S = out.substitutions   # Anzahl ersetzter Wörter
D = out.deletions       # Anzahl fehlender Wörter
I = out.insertions      # Anzahl zusätzlicher Wörter

print(f"Substitutions: {S}")
print(f"Deletions:     {D}")
print(f"Insertions:    {I}")
print(f"WER:           {out.wer:.2%}")

Substitutions: 2
Deletions:     0
Insertions:    1
WER:           21.43%


In [22]:
from difflib import SequenceMatcher

# 1) Preprocessing + Tokenisierung
ref = ref_text.lower().replace(".", "").replace(",", "")  # Beispiel Punctuation-Strip
hyp = hyp_text.lower().replace(".", "").replace(",", "")
ref_tokens = ref.split()
hyp_tokens = hyp.split()

# 2) Alignment und OpCodes
matcher = SequenceMatcher(None, ref_tokens, hyp_tokens)
opcodes = matcher.get_opcodes()
# opcodes: List of (tag, i0, i1, j0, j1)
#   tag ∈ {"equal","replace","delete","insert"}


In [23]:
opcodes

[('replace', 0, 1, 0, 1),
 ('equal', 1, 8, 1, 8),
 ('insert', 8, 8, 8, 9),
 ('equal', 8, 12, 9, 13),
 ('replace', 12, 13, 13, 14),
 ('equal', 13, 14, 14, 15)]

In [24]:
medical_entities = {"nausea","puls","80","aspirin"}

S_med = 0
D_med = 0
I_med = 0

for tag, i0, i1, j0, j1 in opcodes:
    # Referenz-Wörter für replace/delete
    for term in ref_tokens[i0:i1]:
        if term in medical_entities:
            if tag == "replace":
                S_med += 1
            elif tag == "delete":
                D_med += 1
    # Hypo-Wörter für insert
    if tag == "insert":
        for term in hyp_tokens[j0:j1]:
            if term in medical_entities:
                I_med += 1

print("Medical Substitutions:", S_med)
print("Medical Deletions:    ", D_med)
print("Medical Insertions:   ", I_med)


Medical Substitutions: 1
Medical Deletions:     0
Medical Insertions:    0


In [26]:
from difflib import SequenceMatcher

# 1) Beispieltexte
ref = "ich habe nausea und ein puls von 80 und heute morgen ein aspirin genommen"
hyp = "iik habe nauseanauseai nausea und ein puls von 80 90 und heute morgen ein arpirin genommen"

# 2) Preprocessing & Tokenisierung
ref_tokens = ref.lower().replace(",", "").replace(".", "").split()
hyp_tokens = hyp.lower().replace(",", "").replace(".", "").split()

# 3) Alignment und OpCodes
matcher = SequenceMatcher(None, ref_tokens, hyp_tokens)
opcodes = matcher.get_opcodes()
# opcodes ist eine Liste von Tuples: (tag, i0, i1, j0, j1)

# 4) Extraktion der Wörter je Operation
subs = []  # Substitutionen: Liste von (ref_word, hyp_word)
dels = []  # Deletionen: Liste von ref_word
ins = []   # Insertions: Liste von hyp_word

for tag, i0, i1, j0, j1 in opcodes:
    if tag == "replace":
        # Für jedes ersetzte Wortpaar
        for r, h in zip(ref_tokens[i0:i1], hyp_tokens[j0:j1]):
            subs.append((r, h))
    elif tag == "delete":
        # Für jedes gelöschte Referenz-Wort
        for r in ref_tokens[i0:i1]:
            dels.append(r)
    elif tag == "insert":
        # Für jedes zusätzlich eingefügte Hypothesen-Wort
        for h in hyp_tokens[j0:j1]:
            ins.append(h)

# 5) Ausgabe
print("Substitutionen (ref → hyp):", subs)
print("Deletionen (ref fehlt):     ", dels)
print("Insertions (zusätzlich):    ", ins)


Substitutionen (ref → hyp): [('ich', 'iik'), ('aspirin', 'arpirin')]
Deletionen (ref fehlt):      []
Insertions (zusätzlich):     ['nauseanauseai', '90']


In [29]:
from difflib import SequenceMatcher

# Beispieltexte mit Tippfehlern
ref = "ich habe nausea und ein puls von 80 und heute morgen ein aspirin genommen"
hyp = "iik habe apendicitis nausea und ein pols von 80 90 und heute morgen ein arpirin genommen"

# Preprocessing & Tokenisierung
ref_tokens = ref.lower().replace(",", "").replace(".", "").split()
hyp_tokens = hyp.lower().replace(",", "").replace(".", "").split()

# Alignment und OpCodes
matcher = SequenceMatcher(None, ref_tokens, hyp_tokens)
opcodes = matcher.get_opcodes()

# Extraktion der Wörter je Operation
subs = []  # Substitutionen: Liste von (ref_word, hyp_word)
dels = []  # Deletionen: Liste von ref_word
ins = []   # Insertions: Liste von hyp_word

for tag, i0, i1, j0, j1 in opcodes:
    if tag == "replace":
        for r, h in zip(ref_tokens[i0:i1], hyp_tokens[j0:j1]):
            subs.append((r, h))
    elif tag == "delete":
        for r in ref_tokens[i0:i1]:
            dels.append(r)
    elif tag == "insert":
        for h in hyp_tokens[j0:j1]:
            ins.append(h)

# Definierte medizinische Entitäten
medical_entities = {"nausea", "puls", "80", "aspirin"}

# Zählen für mWER
S_med = sum(1 for r, h in subs if r in medical_entities)
D_med = sum(1 for r in dels if r in medical_entities)
I_med = sum(1 for h in ins if h in medical_entities)

# Gesamtzahl der medizinischen Entitäten in ref
N_med = sum(1 for t in ref_tokens if t in medical_entities)

# Berechnung der mWER
mWER = (S_med + D_med + I_med) / N_med if N_med > 0 else 0

# Ausgabe
print(f"Medical Substitutions (S_med): {S_med}")
print(f"Medical Deletions     (D_med): {D_med}")
print(f"Medical Insertions   (I_med): {I_med}")
print(f"Anzahl Medical Terms in ref (N_med): {N_med}")
print(f"mWER: {mWER:.2%}")


Medical Substitutions (S_med): 2
Medical Deletions     (D_med): 0
Medical Insertions   (I_med): 0
Anzahl Medical Terms in ref (N_med): 4
mWER: 50.00%


In [ ]:

_id
6765e8a4a84bdac1bcd5a744

In [37]:
ref_text = "wir sind in der wohnung paramedic schmidt die zugaenglichkeit war unkompliziert gut emt mueller lassen sie uns schnell zu jane gehen sie ist gestuerzt und kann nicht mehr aufstehen verstanden paramedic schmidt wir sollten uns beeilen ja emt mueller wir muessen sicherstellen dass jane in ordnung ist und die notwendige medizinische hilfe erhaelt ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt genau emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja emt mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedic schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja emt mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedic schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja emt mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedic schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt hallo jane mein name ist mueller und das ist mein kollege schmidt wir sind vom rettungsdienst und sind hier um ihnen zu helfen koennen sie uns sagen wo genau sie schmerzen haben ich habe schmerzen in meinem ruecken und ich fuehle mich so schwach okay jane wir werden jetzt ihre vitalfunktionen ueberpruefen und schauen ob sie sich verletzt haben koennten wir werden sie gut versorgen schmidt ich ueberpruefe die atemwege und die atmung koennen sie bitte den kreislauf und das bewusstsein ueberpruefen verstanden mueller jane ich werde jetzt ihren puls und blutdruck messen und schauen wie gut sie bei bewusstsein sind ist das in ordnung fuer sie ja das ist in ordnung jane ihre atemwege sind frei und ihre atmung ist stabil schmidt wie sieht es mit dem kreislauf und dem bewusstsein aus der puls ist stabil der blutdruck ist etwas niedrig aber noch im rahmen jane ist bei vollem bewusstsein und reagiert angemessen gut schmidt jane wir werden jetzt ihre schmerzen genauer beurteilen koennen sie uns auf einer skala von null bis zehn sagen wie stark ihre schmerzen sind wobei null keine schmerzen und zehn die schlimmsten vorstellbaren schmerzen sind ich wuerde sagen es ist eine sieben verstanden jane wir werden jetzt alles tun um ihre schmerzen zu lindern und sie sicher ins krankenhaus zu bringen jane sie haben uns gesagt dass sie medikamente fuer ihr asthma und ihre copd nehmen koennen sie uns sagen welche medikamente das sind ja ich nehme albuterol ich inhaliere es albuterol verstanden das ist ein kurz wirkender beta zweiadrenergischer agonist nehmen sie noch andere medikamente ein nein nur das albuterol haben sie irgendwelche allergien gegen medikamente jane nein ich habe keine medikamentenallergien das ist gut zu wissen und wie sieht es mit ihrer medizinischen vorgeschichte aus gab es in der vergangenheit operationen oder schwere krankheiten nein nichts ernsthaftes das ist gut zu hoeren jane wir werden jetzt alles tun um ihre schmerzen zu lindern und sie sicher ins krankenhaus zu bringen ja wir sind hier um ihnen zu helfen jane wir werden sie gut versorgen jane wir werden jetzt ihre vitalwerte ueberpruefen ist das in ordnung fuer sie ja das ist in ordnung ihre temperatur betraegt siebenundneunzig punkt neun°f herzfrequenz siebenundsechzig schlaege pro minute atemfrequenz neunzehn atemzuege pro minute blutdruck einhunderteinundzwanzig über zweiundsechzig mmhg und sauerstoffsaettigung einhundert alles sieht gut aus jane wir werden ihnen jetzt acetaminophen 325mg tab geben um ihre schmerzen zu lindern ist das in ordnung fuer sie ja danke mueller ich vermute eine moegliche orthopaedische fraktur des kopfes und des halses des rechten femurs wir sollten sie ins krankenhaus bringen um das zu bestaetigen ja das klingt nach dem besten plan jane wir vermuten dass sie eine fraktur im rechten oberschenkel haben koennten wir werden sie ins krankenhaus bringen um das zu bestaetigen ist das in ordnung fuer sie ja das ist in ordnung danke fuer ihre hilfe schmidt wir muessen jane ins krankenhaus bringen ihre vitalwerte sind stabil aber wir muessen sicherstellen dass sie keine fraktur hat ja ich stimme zu jane wir werden sie ins krankenhaus bringen um sicherzustellen dass sie keine schweren verletzungen haben sind sie damit einverstanden ja das ist in ordnung ich vertraue ihnen gut jane wir werden ihre vitalwerte weiterhin ueberwachen um sicherzustellen dass sie stabil bleiben ihre temperatur betraegt derzeit siebenundneunzig punkt neun°f ihre herzfrequenz liegt bei siebenundsechzig schlaegen pro minute ihre atemfrequenz bei neunzehn atemzuegen pro minute ihr blutdruck bei einhunderteinundzwanzig über zweiundsechzig mmhg und ihre sauerstoffsaettigung ist einhundert wir haben ihnen auch acetaminophen 325mg tab gegeben um ihre schmerzen zu lindern lassen sie uns wissen wenn sie immer noch schmerzen haben ich fuehle mich ein bisschen besser danke das ist gut zu hoeren jane wir werden sie jetzt ins krankenhaus bringen dort werden sie weitere untersuchungen durchfuehren um sicherzustellen dass sie keine fraktur haben genau jane wir vermuten eine moegliche orthopaedische fraktur des kopfes und des halses des rechten femurs im krankenhaus werden sie das genauer untersuchen ich verstehe danke dass sie sich um mich kuemmern das ist unser job jane wir sind hier um ihnen zu helfen wir werden sie jetzt ins krankenhaus bringen jane wir haben sie jetzt sicher auf der trage im rettungswagen platziert wir werden sie waehrend der fahrt weiterhin ueberwachen danke ich fuehle mich schon etwas besser das ist gut zu hoeren jane das acetaminophen 325mg tab das wir ihnen gegeben haben sollte ihre schmerzen lindern schmidt koenntest du bitte ihre vitalwerte weiterhin ueberwachen wir muessen sicherstellen dass sie stabil bleiben natuerlich mueller ihre temperatur betraegt siebenundneunzig punkt neun°f herzfrequenz siebenundsechzig schlaege pro minute atemfrequenz neunzehn atemzuege pro minute blutdruck einhunderteinundzwanzig über zweiundsechzig mmhg und sauerstoffsaettigung einhundert alles sieht momentan stabil aus perfekt schmidt jane wir vermuten dass sie eine orthopaedische fraktur des kopfes und des halses des rechten femurs haben koennten im krankenhaus werden wir das genauer untersuchen lassen ich verstehe ich danke ihnen fuer ihre hilfe wir sind fuer sie da jane wir werden sie weiterhin ueberwachen und sicherstellen dass sie sicher im krankenhaus ankommen frau schmidt ich denke wir sollten jane ins st marien krankenhaus bringen sie sind gut ausgestattet fuer orthopaedische verletzungen ja das klingt nach einer guten entscheidung ich werde die transportfaehigkeit sicherstellen jane wir werden sie jetzt ins krankenhaus bringen okay ja das ist in ordnung ich fuehle mich etwas besser das ist gut zu hoeren jane wir haben ihnen acetaminophen gegeben das sollte ihre schmerzen lindern ihre vitalwerte sind stabil das ist ein gutes zeichen ich werde das krankenhaus informieren dass wir unterwegs sind und was sie erwarten koennen st marien hier ist rettungswagen zweiundvierzig wir bringen eine patientin mit verdacht auf orthopaedische fraktur des kopfes und des halses des rechten femurs vitalwerte sind stabil patientin ist ansprechbar jane wir sind gleich da sie sind in guten haenden jane wir heben sie jetzt auf die trage wir werden versuchen sie so wenig wie moeglich zu bewegen um weitere schmerzen zu vermeiden okay danke mueller ich halte den kopf du nimmst die beine verstanden schmidt auf drei eins zwei drei autsch das tut weh tut mir leid jane wir versuchen so vorsichtig wie moeglich zu sein wir sind gleich im rettungswagen jane sie sind jetzt sicher auf der trage wir bringen sie jetzt in den rettungswagen okay danke mueller ich gehe voran und mache den weg frei gut schmidt ich folge dir mit jane jane wir sind jetzt im rettungswagen wir fahren gleich los okay danke ich fuehle mich etwas besser seit ich das acetaminophen genommen habe das ist gut zu hoeren jane wir ueberwachen weiterhin ihre vitalwerte und sind gleich im st marien krankenhaus mueller ich bereite die infusion vor um jane weiterhin mit fluessigkeit zu versorgen gut schmidt ich ueberwache weiterhin janes vitalwerte jane wie fuehlen sie sich jetzt haben die schmerzen nachgelassen ja ein bisschen es ist immer noch unangenehm aber es ist besser das ist gut zu hoeren jane wir haben ihre vitalwerte im blick ihr blutdruck und ihre herzfrequenz sind stabil schmidt bereiten sie bitte eine infusion vor wir muessen sicherstellen dass jane genug fluessigkeit bekommt verstanden mueller ich bereite die infusion vor jane wir legen ihnen jetzt eine infusion an um sie mit fluessigkeit zu versorgen das wird ihnen helfen sich besser zu fuehlen okay danke die infusion laeuft mueller ich habe das st marien krankenhaus informiert sie erwarten uns in etwa fünfzehn minuten gut schmidt ich ueberwache weiterhin janes vitalwerte jane wir sind auf dem weg ins krankenhaus sie sind in guten haenden danke ich fuehle mich schon etwas besser das ist gut zu hoeren jane wir sind gleich im krankenhaus halten sie durch schmidt halten sie das krankenhaus bitte auf dem laufenden ueber janes zustand und unsere eta verstanden mueller ich informiere das krankenhaus jane wie fuehlen sie sich jetzt haben die schmerzen weiter nachgelassen ja es ist etwas besser aber ich fuehle mich immer noch nicht ganz wohl das ist normal jane die medikamente wirken weiter und wir sind bald im krankenhaus dort koennen die aerzte sie genauer untersuchen schmidt wie sieht es mit den vitalwerten aus blutdruck und herzfrequenz sind stabil die infusion laeuft auch gut gut dann informiere bitte das krankenhaus ueber unseren aktuellen status und die voraussichtliche ankunftszeit verstanden mueller ich rufe jetzt im st marien krankenhaus an und gebe die informationen durch jane wir haben alle notwendigen massnahmen getroffen sie sind stabil und wir sind bald im krankenhaus versuchen sie ruhig zu bleiben ich versuche es danke dass sie sich so gut um mich kuemmern schmidt bitte informiere die klinik ueber unsere ankunft und den zustand von jane verstanden mueller hallo hier ist der rettungsdienst wir haben eine patientin namens jane die ueber schmerzen klagt die schmerzen haben etwas nachgelassen sie fuehlt sich aber immer noch unwohl ihre vitalzeichen sind stabil wir haben eine infusion gelegt wir sind in etwa zehn minuten bei ihnen jane wir sind gleich im krankenhaus dort wird man sich gut um sie kuemmern und sie genauer untersuchen danke das beruhigt mich die klinik ist informiert und bereitet sich auf unsere ankunft vor mueller gut gemacht schmidt jane wir sind jetzt gleich da ich danke ihnen beiden sie haben mir sehr geholfen das ist unser job jane wir sind froh dass wir helfen konnten mueller nach der uebergabe sollten wir den einsatz nochmal besprechen gute idee schmidt es ist immer wichtig unsere arbeit zu reflektieren und zu schauen wo wir uns verbessern koennen genau mueller so koennen wir sicherstellen dass wir unseren patienten die bestmoegliche versorgung bieten"

In [38]:
hyp_text = "wir sind in der wohnung paramedicschmidt die zugaenglichkeit war unkompliziert gut der md mueller lassen sie uns schnell zu janne gehen sie ist gestuerzt und kann nicht mehr aufstehen verstanden paramedicschmidt wir sollten uns beeilen ja der md mueller wir muessen sicherstellen dass janne in ordnung ist und die notwendige medizinische hilfe erhaelt ja paramedicschmidt wir sind hier um janne zu helfen und sicherzustellen dass sie die besprechliche medizinische versorgung erhaelt genau md mueller lassen sie uns jetzt zu janne gehen und ihre situation beurteilen ja paramedicschmidt wir sollten uns beeilen und janne die notwendige medizinische hilfe erhaelt ja md mueller wir sind hier um janne zu helfen und sicherzustellen ja sendsin dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedicschmidt lassen sie uns jetzt zu janne gehen und ihre situation beurteilen ja mandim mueller wir sollten uns beeilen und janne die notwendige medizinische hilfe erhaelt ja paramedicschmidt wir sind hier um janne zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedicschmidt wir muessen sicherstellen dass sie die besprechliche medizinische versorgung erhaelt uns jetzt zu jane gehen und ihre situation beurteilen ja pfarrer medicschmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rehentemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja pfarrer medicschmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rehentemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja pfarrer medicschmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja rehentemula lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja pfarrer medicschmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rehentemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja pfarrer medicschmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rehentemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja pfarrer medicschmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja rehentemula lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja pfarrer medicschmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rehentemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja pfarrer medicschmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rehentemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja pfarrer medicschmidt wir sind hier zu jane gehen und ihre situation beurteilen ja rehentemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja pfarrer medicschmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja rehentemula lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja pfarrer medicschmidt gefoerdert finanziert und so weiter ja ob das gut wird fuer sie oder ist das vollkommen einfach ja ob das gut soll dann muss kein fehler daran sein komm komm komm automate und safety komm ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja herr demueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja herr demueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja herr demueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja herr demueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja herr demueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja herr demueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja herr demueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja herr demueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja herr demueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja herr demueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja herr demueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja herr demueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja herr demueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja herr demueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja herr demueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja herr demueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja herr demueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja herr demueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja herr demueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja herr demueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns bei ari und jane die notwendige medizinische hilfe geben ja wmd mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja wmd mueller wir sollten uns bei ari und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja wmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns bei ari und jane die notwendige medizinische hilfe geben ja wmd mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja wmd mueller wir sollten uns bei ari und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja wmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns bei ari und jane die notwendige medizinische hilfe geben ja wmd mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja wmd mueller wir sollten uns jetzt zu jane gehen und ihre situation beurteilen ja wmd mueller wir sollten uns bei ari und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja wmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja wmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt sie sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rmd mueller sie sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja rmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja rmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rmd mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja rmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rmd mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rmd mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja rmd mueller lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane die notwendige medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja rmd mueller wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane die notwendige medizinische hilfe geben ja rmd mueller wir sind hier um jane die notwendige medizinische versorgung erhaelt ja rmd mueller wir sind hier um jane die notwendige medizinische versorgung erhaelt ja rmd mueller wir sind hier um jane die notwendige medizinische versorgung erhaelt ja remdemula lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja remdemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja remdemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja remdemula lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja remdemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja remdemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja remdemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja remdemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja remdemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja paramedik schmidt wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja remdemula lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja remdemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja remdemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedik schmidt lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja remdemula wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja remdemula wir sind hier um jane zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt ja remdemula lassen sie uns jetzt zu jane gehen und ihre situation beurteilen ja paramedik schmidt wir sollten uns beeilen und jane die notwendige medizinische hilfe geben ja remdemula wir sind hier um jane die notwendige medizinische hilfe geben ja remdemula wir sind hier um jane die notwendige medizinische hilfe geben dass sie die bestmoegliche medizinische versorgung erhaelt ja paramedikstaet lassen sie uns jetzt zu jan gehen und ihre situation urteilen ja herr dimmler wir sollten uns beeilen und jan in die notwendige medizinische hilfe geben ja paramedikstaet wir sind hier um jan zu helfen und sicherzustellen dass sie die bestmoegliche medizinische versorgung erhaelt hallo jan mein name ist mueller und das ist mein kollege schmidt wir sind vom rettungsdienst und sind hier um ihnen zu helfen koennen sie uns sagen wo genau sie schmerzen haben ich habe schmerzen im ruecken und ich fuehle mich so schlauchig okay jan wir werden jetzt ihre vitalfunktion ueberpruefen und schauen ob sie sich verletzt haben koennen wir werden sie gut versorgen schmidt ich ueberpruefe die arme sie haben atemwege und sie atmen koennen sie bitte den kreislauf und das bewusstsein ueberpruefen verstanden mueller jan ich werde jetzt ihren kurs und eindruck messen und schauen wie sie bei bewusstsein sind ist das ein atem fuer sie ja das ist in ordnung jan ihre atemwege sind frei und ihre atmung ist staerker schmidt wie sieht es mit dem kreislauf und dem bewusstsein aus ja das ist in ordnung der druck ist stabil der druck ist etwas niedrig aber noch im rahmen jan ist bei vollem bewusstsein und reagiert angemessen gute schmidt jan wir werden jetzt ihre schmerzen genauer urteilen koennen sie uns auf einer skala von null bis zehn sagen wie stark ihre schmerzen sind wobei null keine schmerzen und zehn die schlimmsten vorstellbaren schmerzen sind ich will sagen es ist eine siege ja das ist eine siege verstanden jan wir werden jetzt alles tun um ihre schmerzen zu lindern und sie sicher ins krankenhaus zu bringen jan sie haben uns gesagt dass sie medikamente fuer ihr asthma und ihre kopf nehmen koennen sie uns sagen welche medikamente das sind ja ich nehme albuterol ich inhaliere es albuterol verstanden das ist ein kurzwirkender betazweiadrenergieabonist nehmen sie noch andere medikamente ein nein nur das albuterol haben sie irgendwelche allergien gegen medikamente jan nein ich habe keine medikamente allergien das ist gut zu wissen und wie sieht es mit ihrer medizinischen vorgeschichte aus gab es in der vergangenheit operationen oder schwere krankheiten nein nichts ernsthaftes das ist gut zu hoeren jan wir werden jetzt alles tun um ihre schmerzen zu lindern und sie sicher ins krankenhaus zu bringen ja wir sind hier um ihnen zu helfen jan wir werden sie gut versorgen jan wir werden jetzt ihre vitawerte ueberpruefen ist das in ordnung fuer sie ja das ist in ordnung ihre temperatur betraegt siebenundneunzig punkt neun grad auf der frequenz siebenundsechzig kilo pro minute und die frequenz neunzehn attensuebe pro minute mit einhunderteinundzwanzig punkt zweiundsechzig invertie sie koennen sich in der pflege nicht mehr verletzen sie koennen sich in der pflege nicht mehr verletzen sie koennen sich in der pflege nicht mehr verletzen sie koennen sich in der pflege nicht mehr verletzen sie koennen sich in der pflege nicht mehr verletzen mit neunundzwanzig punkt dreihundertzehn invertleten im saugeschmack wird die druck einhundert prozent alles sieht gut aus mit neunundzwanzig punkt dreihundertzehn invertleten im saugeschmack wird die druck einhundert prozent alles sieht gut aus jan wir werden ihnen jetzt akzenterin dreihundertfünfundzwanzig mgthp geben um ihre schmerzen zu lindern ist das in ordnung fuer sie jan wir werden ihnen jetzt akzenterin dreihundertfünfundzwanzig mgthp geben um ihre schmerzen zu lindern ist das in ordnung fuer sie ja danke general ich vermute eine moegliche orthopaedische fraktur des korpuses und des halses des rechtlichen komponents wir sollten sie ins krankenhaus bringen um das zu bestaetigen ja das klingt nach dem besten plan jene wir vermuten dass sie eine faktur im rechten oberschenkel haben koennten wir werden sie ins krankenhaus bringen um das zu bestaetigen ist das in ordnung fuer sie ja das ist in ordnung danke fuer ihre hilfe schmidt wir muessen jane ins krankenhaus bringen ihre widerfallwerte sind stabil aber wir muessen nicht gesteigen dass sie keine faktur hat ja ich stimme zu jane wir werden sie ins krankenhaus bringen um sicherzustellen dass sie keine schweren verletzungen haben sind sie damit einverstanden ja das ist in ordnung ich vertraue ihnen gut jane wir werden ihre widerfallwerte weiterhin ueberwachen um sicherzustellen dass sie stabil bleiben ihre temperatur betraegt derzeit sieben komma neunzigneun grad f ihre herzfrequenz liegt bei sieben komma sechzig ihre atemfrequenz liegt bei neunzehn atemzuegen pro minute ihr blutdruck bei einhunderteins komma zweihundertzweiundzwanzig im mhg und ihre sauerstofflaettigung einhundert wir haben ihnen auch acetaminol dreihundertfünfundzwanzig im gtab gegeben um ihre schmerzen zu lindern lassen sie uns wissen wenn sie immer noch schmerzen haben ich fuehle mich ein bisschen besser danke das ist gut zu hoeren jane wir werden sie jetzt ins krankenhaus bringen dort werden sie weitere untersuchungen durchfuehren um sicherzustellen dass sie keine faktur haben genau jane wir vermuten eine moegliche orthopaedische fraktur des kopfes und des halses des rechten hinters im krankenhaus werden sie das genauer untersuchen ich verstehe danke dass sie sich um mich kuemmern das ist unser job jane wir sind hier um ihnen zu helfen wir werden sie jetzt ins krankenhaus bringen jane wir haben sie jetzt sicher auf der trage im rettungswagen platziert wir werden sie waehrend der fahrt weiterhin ueberwachen danke ich fuehle mich schon etwas besser das ist gut zu hoeren jane das acetaminol dreihundertfünfundzwanzig im gtab das sie ihnen gegeben haben sollte ihre schmerzen lindern schmidt koenntest du bitte ihre futterwerte weiterhin ueberwachen ja wir muessen sicherstellen dass sie stabil bleiben natuerlich mueller ihr temperaturbetrieb siebenundneunzig komma neun°c herzfrequenz siebenundsechzig komma vier°c herzfrequenz achtundneunzig komma fünf°c mitdruck einhunderteinundzwanzig komma zweiundsechzig mhzsauerstofffertigung einhundert alles sieht momentan stabil aus perfekt schmidt jane wir vermuten dass sie eine orthopaedische fraktur des kopfes haben wir werden sie weiterhin ueberwachen und sicherstellen dass sie sicher im krankenhaus ankommen im krankenhaus werden wir das genauer untersuchen lassen ich verstehe ich danke ihnen fuer ihre hilfe wir sind fuer sie da jane wir werden sie weiterhin ueberwachen und sicherstellen dass sie sicher im krankenhaus ankommen frau schmidt ich denke wir wuerden ihnen jetzt mal in ein krankenhaus bringen sie sind gut ausgestattet fuer orthopaedische verletzungen ja das geht nach einer guten entscheidung ich werde die transportfaehigkeit sicherstellen jane wir werden sie jetzt ins krankenhaus bringen okay ja das ist in ordnung ich fuehle mich etwas besser das ist gut zu hoeren jane wir haben ihnen acetaminofen gegeben das sollte ihre schmerzen mindern ihre vitalwerte sind stabil das ist ein gutes zeichen ich werde das krankenhaus informieren dass wir unterwegs sind und was sie erwarten koennen ja das ist gut wir bringen eine patentenmitverdachter auf orthopaedische fraktur des kopfes und das heisst es ist das rechten fuer bos vitalwerte sind stabil patenten ist ansprechbar jane wir sind gleich da sie sind den guten hinten jane wir heben sie jetzt auf die trage wir werden versuchen sie so wenig wie moeglich zu bewegen um weitere schmerzen zu vermeiden okay danke mueller ich halte den kopf auf du nimmst die beine verstanden schritt auf drei eins zwei drei autsch das tut weh tut mir leid jane wir versuchen so vorsichtig wie moeglich zu sein wir sind gleich in rettung zu kommen jane sie sind jetzt nicht auf der trage wir melden sie jetzt in den rettungswagen okay danke mueller ich gehe voran und mache den weg frei schmidt ich folge dir mit jane jane wir sind jetzt in rettungswagen wir fahren gleich los okay danke ich fuehle mich etwas besser seit ich das acetamolfuellung behabe das ist gut so jane wir ueberwachen weiterhin die vitalwerte und sind gleich im sinn neu in krankem haus mueller ich bereite die inklusion vor um jane weiterhin mit fluessigkeit zu versorgen gut schmidt ich ueberwache weiterhin die vitalwerte jane wie fuehlen sie sich jetzt haben die schmerzen nachgelassen ja ein bisschen es ist immer leicht unangenehm aber es ist besser das ist gut zu hoeren jane wir haben ihre vitalwerte im blick ihr blutdruck und ihre herzfrequenz sind stabil schmidt bereiten sie bitte eine infusion vor wir muessen sicherstellen dass jane genug fluessigkeit bekommt verstanden mueller ich bereite die infusion vor jane wir legen ihnen jetzt eine infusion an um sie mit fluessigkeit zu versorgen das wird ihnen helfen sich besser zu fuehlen okay danke die infusion laeuft mueller ich habe das letzte neu in krankem haus in form sie erwarten uns in etwa fünfzehn minuten gut schmidt ich ueberwache weiterhin die antizitalwerte jane wir sind auf dem weg ins kranken haus sie sind in gutem haenden danke ich fuehle mich schon etwas besser das ist gut zu hoeren jane wir sind gleich im kranken haus halten sie sich schmidt erhalten sie das kranken haus bitte auf dem laufenden weile in zustand und unwerte verstanden mueller ich informiere das kranken haus jane wie fuehlen sie sich jetzt haben die schmerzen weiter nachgelassen ja es ist etwas besser aber ich fuehle mich immer noch nicht ganz so gut das ist normal jane die medikamente werden weiter und wir sind bald im kranken haus dort koennen die aerzte sie genauer untersuchen schmidt wie sieht es mit ihnen wieder in der apnose blutdruck und herzfrequenz sind stabil die infusion laeuft auch gut gut dann informiere bitte das kranken haus ueber unseren aktuellen status und die vorbereitung wir haben eine voraussichtliche ankunftszeit verstanden mueller ich rufe jetzt die maestee ich nenne ihnen das krankenhaus an und gebe die informationen durch jane wir haben alle notwendigen massnahmen getroffen sie sind stabil und wir sind bald im kranken haus versuchen sie ruhig zu bleiben ich versuche es danke dass sie sich so gut um mich kuemmern schmidt er bitte informiere die klinik ueber unseren ankunfts und zustand von jane verstanden mueller hallo hier ist der rettungsdienst wir haben eine patientin namens jane die ueber schmerzen fragt die schmerzen haben etwas nachgelassen sie fuehlt sich aber immer noch unwohl ihre vitalzeichen sind stabil wir haben eine infusion gelegt wir sind in etwa zehn minuten bei ihnen ja wir sind gleich im kranken haus dort wird man sich gut um sie kuemmern und sie genauer untersuchen danke das beruhigt mich die klinik ist informiert und bereitet sich auf unsere ankuenfte frau mueller gut gemacht schmidt werner wir sind jetzt gleich da ich danke ihnen beiden sie haben mir sehr geholfen das ist unser job ja na ja wir sind froh dass wir helfen konnten mueller nach der uebergabe sollten wir den einsatz nochmal besprechen gute idee schmidt es ist immer wichtig unsere arbeit zu reflektieren und zu schauen ob wir uns verbessern koennen genau mueller so koennen wir sicherstellen dass wir unseren patienten die bestmoegliche versorgung bieten"

In [ ]:

_id
6765e8aaa84bdac1bcd5a7a1


In [41]:
ref_text = "julia wir haben einen fünfundvierzigjaehrigen mann mit uebelkeit durchfall und schwaeche die adresse ist musterstrasse einhundertdreiundzwanzig in musterstadt der patient befindet sich in seiner wohnung im ersten stock verstanden michael keine hinweise auf ein gefaehrdungspotential richtig richtig julia zugang zur wohnung ist ueber das treppenhaus okay michael dann bereiten wir uns auf einen patienten mit moeglicher dehydration vor ich nehme an wir werden eine infusion vorbereiten muessen ja das klingt nach einem guten plan julia lass uns das material fuer die infusion und die vitalzeichenmessung bereitstellen guten tag mein name ist michael bauer und das ist meine kollegin julia schmidt wir sind vom rettungsdienst wie koennen wir ihnen helfen mir ist uebel und ich habe durchfall ich fuehle mich schwach wir werden sie jetzt untersuchen zuerst pruefen wir ihre atemwege die atemwege sind frei wie oft atmen sie pro minute ich weiss nicht genau ich zaehle zwanzig atemzuege pro minute das ist etwas erhoeht aber noch im stabilen bereich wir messen jetzt ihren blutdruck und ihre herzfrequenz okay ihr blutdruck betraegt einhundertfünfundfünfzig über fünfundachtzig mmhg und ihre herzfrequenz liegt bei dreiundneunzig schlaegen pro minute das ist stabil haben sie irgendwelche neurologischen beschwerden koennen sie alle meine finger sehen ja ich kann alle ihre finger sehen gut es scheint keine neurologischen defizite zu geben wie fuehlen sie sich ist ihnen kalt oder warm mir ist nicht kalt und auch nicht warm die umgebungstemperatur und ihr zustand scheinen unauffaellig zu sein wir werden jetzt eine infusion vorbereiten um ihren fluessigkeitshaushalt zu stabilisieren ja das ist eine gute idee ich werde das material fuer die infusion und die vitalzeichenmessung bereitstellen herr mueller sie sagten sie haben seit einigen tagen durchfall und fuehlen sich schwach haben sie noch andere beschwerden nein nur diese uebelkeit und durchfall ich fuehle mich einfach schlapp haben sie blut im stuhl bemerkt herr mueller nein das habe ich nicht sie haben uns gesagt dass sie isosorbidmononitrat einnehmen haben sie heute morgen ihre uebliche dosis eingenommen ja das habe ich ich nehme es jeden morgen wegen meiner herzprobleme haben sie noch andere medikamente eingenommen herr mueller nein nur das isosorbidmononitrat okay danke fuer die informationen wir werden jetzt eine infusion vorbereiten um ihren fluessigkeitshaushalt zu stabilisieren julia koenntest du das bitte uebernehmen ja ich bereite das material fuer die infusion vor herr mueller wir werden ihnen jetzt eine infusion geben um ihren fluessigkeitshaushalt zu stabilisieren ist das in ordnung fuer sie ja das ist in ordnung danke herr mueller wir haben sie jetzt an ein ueberwachungsgeraet angeschlossen es misst ihre sauerstoffsaettigung herzfrequenz blutdruck und temperatur ihre sauerstoffsaettigung liegt bei zweiundneunzig das ist etwas niedrig ihre herzfrequenz betraegt dreiundneunzig schlaege pro minute das ist im normalen bereich ihr blutdruck ist mit einhundertfünfundfünfzig über fünfundachtzig mmhg etwas erhoeht und ihre temperatur liegt bei achtundneunzig punkt null°f das ist normal herr mueller fuehlen sie sich sehr unwohl oder haben sie schmerzen ich fuehle mich schwach und mir ist uebel aber schmerzen habe ich keine gut herr mueller wir haben keine aeusserlichen verletzungen oder prellmarken bei ihnen festgestellt ihre haut ist normal gefaerbt und gut durchblutet julia ich denke wir sollten herrn mueller aufgrund seiner symptome und der niedrigen sauerstoffsaettigung ins krankenhaus bringen dort kann er weiter untersucht und behandelt werden ja das sehe ich auch so herr mueller wir werden sie jetzt ins krankenhaus bringen dort koennen die aerzte sie weiter untersuchen und behandeln ist das in ordnung fuer sie ja das ist in ordnung ich hoffe es ist nichts ernstes wir tun unser bestes um ihnen zu helfen herr mueller im krankenhaus werden sie gut versorgt werden herr mueller wir werden sie jetzt ins krankenhaus bringen dort koennen die aerzte weitere untersuchungen durchfuehren und die ursache ihrer beschwerden genauer feststellen in ordnung ich verstehe ich werde ihnen jetzt einen intravenoesen zugang legen herr mueller das ist fuer den fall dass wir ihnen unterwegs fluessigkeit oder medikamente verabreichen muessen okay machen sie das julia wie sieht es mit der sauerstoffsaettigung aus sollten wir ihm zusaetzlich sauerstoff geben gute idee michael die sauerstoffsaettigung ist immer noch etwas niedrig herr mueller wir werden ihnen jetzt zusaetzlich sauerstoff geben das sollte ihnen helfen sich besser zu fuehlen alles klar danke julia ich denke wir sollten auch den blutdruck im auge behalten er ist immer noch etwas erhoeht ja das sollten wir wir werden ihren blutdruck herr mueller waehrend der fahrt weiterhin ueberwachen verstanden danke dass sie sich um mich kuemmern das ist unser job herr mueller wir sind gleich im krankenhaus halten sie durch julia wie sieht es mit den vitalwerten aus blutdruck ist immer noch etwas erhoeht einhundertfünfundvierzig über neunzig aber stabil sauerstoffsaettigung liegt bei zweiundneunzig wir geben weiterhin sauerstoff gut halten wir das so bei herr mueller wie fuehlen sie sich ich bin etwas muede aber es geht das ist normal herr mueller wir sind gleich im krankenhaus und dort wird man sich gut um sie kuemmern michael ich denke wir sollten den blutdruck weiterhin engmaschig kontrollieren er ist zwar stabil aber immer noch erhoeht ja das sehe ich auch so lass uns alle fünfzehn minuten messen herr mueller wir werden ihren blutdruck weiterhin engmaschig kontrollieren um sicherzustellen dass er stabil bleibt ist das in ordnung fuer sie ja das ist in ordnung danke dass sie sich so gut um mich kuemmern das ist unser job herr mueller wir sind gleich im krankenhaus julia ich denke wir sollten herrn mueller in die klinik st josef bringen die haben eine gute internistische abteilung und koennen dort die notwendigen untersuchungen durchfuehren ja das klingt nach einer guten entscheidung ich werde die klinik informieren und den transport vorbereiten herr mueller wir bringen sie jetzt in die klinik st josef dort haben sie eine spezialisierte abteilung die sich um ihre symptome kuemmern kann okay danke ich fuehle mich so muede das ist verstaendlich herr mueller sie sind in guten haenden und wir tun alles um ihnen zu helfen julia koennen wir herrn mueller im liegen transportieren sein zustand scheint stabil genug zu sein ja das sollte gehen ich werde die trage vorbereiten herr mueller wir werden sie jetzt auf die trage legen und ins krankenhaus bringen sie koennen dabei liegen bleiben okay danke julia ich habe herrn mueller auf die trage gelegt koennen wir ihn jetzt in den rettungswagen bringen ja michael ich habe bereits alles im rettungswagen vorbereitet lass uns herrn mueller jetzt transportieren herr mueller wir werden sie jetzt in den rettungswagen bringen sie liegen auf einer trage und wir werden sie vorsichtig transportieren ist das in ordnung fuer sie ja das ist in ordnung ich fuehle mich nur so muede das ist normal herr mueller sie sind krank und ihr koerper braucht ruhe wir bringen sie jetzt in die klinik wo sie die noetige behandlung bekommen julia ich werde herrn mueller in den rettungswagen schieben bitte ueberwache seine vitalzeichen waehrend des transports verstanden michael ich habe das ueberwachungsgeraet bereits angeschlossen und werde seine vitalzeichen kontinuierlich ueberwachen herr mueller wir sind jetzt im rettungswagen julia wird ihre vitalzeichen ueberwachen um sicherzustellen dass es ihnen gut geht danke ich fuehle mich sicherer wenn ich weiss dass sie mich ueberwachen das ist unser job herr mueller wir sind hier um sicherzustellen dass es ihnen gut geht wir fahren jetzt los zur klinik st josef julia wie sehen die vitalwerte von herrn mueller aus die vitalwerte sind stabil michael der blutdruck ist bei einhundertzwanzig über achtzig mmhg die herzfrequenz liegt bei achtzig schlaegen pro minute und die sauerstoffsaettigung ist bei achtundneunzig es gibt keine anzeichen von atemnot oder schmerzen das klingt gut julia herr mueller wie fuehlen sie sich haben sie schmerzen oder unwohlsein nein ich fuehle mich eigentlich ganz gut nur ein bisschen muede das ist normal herr mueller ihre muedigkeit ist aufgrund ihrer krankheit wir sind gleich in der klinik st josef und dort werden sie die benoetigte behandlung erhalten julia bitte behalte die vitalwerte im auge und informiere mich wenn es veraenderungen gibt natuerlich michael ich behalte die ueberwachungsgeraete im blick herr mueller wir sind gleich da sie sind in guten haenden wir werden sie gut versorgen julia wie sieht es aus sind die vitalwerte von herrn mueller immer noch stabil ja michael der blutdruck ist immer noch bei einhundertzwanzig über achtzig mmhg die herzfrequenz bei achtzig schlaegen pro minute und die sauerstoffsaettigung bei achtundneunzig keine veraenderungen sehr gut julia herr mueller wie fuehlen sie sich haben sie schmerzen oder atemnot nein keine schmerzen oder atemnot ich fuehle mich nur sehr muede das ist normal herr mueller sie sind krank und ihr koerper braucht ruhe wir sind bald in der klinik st josef und dort werden sie die benoetigte behandlung erhalten michael ich werde weiterhin die vitalwerte im auge behalten und sie ueber veraenderungen informieren danke julia das ist sehr wichtig herr mueller wir sind fuer sie da wenn sie etwas brauchen sagen sie uns bitte bescheid danke ich werde das tun leitstelle hier rettungswagen drei wir haben einen fünfundvierzigjaehrigen maennlichen patienten mit verdacht auf melena vitalwerte sind stabil ankunft in klinik st josef in etwa fünfzehn minuten bitte bereiten sie sich auf diagnostische verfahren vor verstanden rettungswagen drei wir informieren die klinik st josef gute fahrt danke leitstelle ende herr mueller wir sind gleich in der klinik dort wird man sie weiter untersuchen okay danke guten tag wir bringen herrn mueller fünfundvierzig jahre alt mit verdacht auf melena er ist ansprechbar und klagt ueber muedigkeit hat aber keine schmerzen oder atemnot die vitalwerte sind stabil vielen dank fuer die information wir uebernehmen jetzt julia das war ein guter einsatz du hast die vitalwerte gut im blick gehabt danke michael ich denke wir haben alles richtig gemacht hoffentlich geht es herrn mueller bald besser"

In [42]:
hyp_text = "julia wir haben einen fünf und vierzigjaehrigen mann mit uebelkeit durchfall und schwaeche die adresse ist musterstrasse einhundertdreiundzwanzig in musterstadt der patient befindet sich in seiner wohnung im ersten stock verstanden michael keine hinweise auf ein gefaehrdungspotential richtig richtig julia zugang zur wohnung ist ueber das treppenhaus okay michael dann bereiten wir uns auf einen patienten mit moeglicher dehydration vor ich nehme an wir werden eine infusion vorbereiten muessen ja das klingt nach einem guten plan julia lass uns das material fuer die infusion und die vitalzeichenmessung bereitstellen guten tag mein name ist michael bauer und das ist meine kollegin julia schmidt wir sind vom rettungsdienst wie koennen wir ihnen helfen mir ist uebel und ich habe durchfall ich fuehle mich schwach wir werden sie jetzt untersuchen zuerst pruefen wir ihre atemwege die atemwege sind frei wie oft atmen sie pro minute ich weiss nicht genau ich zaehle zwanzig atemzuege pro minute das ist etwas erhoeht aber noch im stabilen bereich wir messen jetzt ihren blutdruck und ihre herzfrequenz guck ich ja ihr blutdruck betraegt einhundertfünfundfünfzig komma fünfundachtzig mmhg und ihre herzfrequenz liegt bei dreiundneunzig schlaegen pro minute das ist stabil haben sie irgendwelche neurologischen beschwerden koennen sie alle meine finger sehen ja ich kann alle ihre finger sehen gut es scheint keine neurologischen defizite zu geben wie fuehlen sie sich ist ihnen kalt oder warm mir ist nicht kalt ich bin kalt und auch nicht warm die umgebungstemperatur und ihr zustand scheinen unauffaellig zu sein wir werden jetzt eine infusion vorbereiten um ihren fluessigkeitshaushalt zu stabilisieren ja das ist eine gute idee ich werde das material fuer die infusion und die vitalzeichenmessung bereitstellen herr mueller sie sagten sie haben seit einigen tagen durchfall und fuehlen sich schwach haben sie noch andere beschwerden nein nur diese uebelkeit und durchfallen ich fuehle mich einfach schlack haben sie blut im stuhl gemerkt herr mueller nein das habe ich nicht sie haben uns gesagt dass sie isosorbidmononitrat einnehmen haben sie heute morgen ihre uebliche dosis eingenommen ja das habe ich ich nehme es jeden morgen wegen meiner herzprobleme haben sie noch andere medikamente eingenommen herr mueller nein nur das isosorbidmononitrat okay danke fuer die informationen wir werden jetzt eine infusion vorbereiten um ihren fluessigkeitshaushalt zu stabilisieren julia koenntest du das bitte uebernehmen ja ich bereite das material fuer die infusion vor herr mueller wir werden ihnen jetzt eine infusion geben um ihren fluessigkeitshaushalt zu stabilisieren ist das in ordnung fuer sie ja das ist in ordnung danke herr mueller wir haben sie jetzt an ein ueberwachungsgeraet angeschlossen es misst ihre sauerstoffsaettigung herzfrequenz blutdruck und temperatur ihre sauerstoffsaettigung liegt bei zweiundneunzig prozent das ist etwas niedrig ihre herzfrequenz betraegt dreiundneunzig schlaege pro minute das ist im normalen bereich ihr blutdruck ist mit einhundertfünfundfünfzig komma fünfundachtzig mmhg etwas erhoeht ihre temperatur liegt bei achtundneunzig komma null grad f das ist normal herr mueller fuehlen sie sich sehr unwohl oder haben sie schmerzen ich fuehle mich schwach und mir ist uebel aber schmerzen habe ich keine gut herr mueller wir haben keine aeusserlichen verletzungen oder prellmarken bei ihnen festgestellt ihre haut ist normal gefaerbt und gut durchblutet julia ich denke wir sollten herrn mueller anrufen er kann aufgrund seiner symptome und der niedrigen sauerstoffsaettigung ins krankenhaus bringen dort kann er weiter untersucht und behandelt werden ja das sehe ich auch so herr mueller wir werden sie jetzt ins krankenhaus bringen dort koennen die aerzte sie weiter untersuchen und behandeln ist das in ordnung fuer sie ja das ist in ordnung ich hoffe es ist nichts ernstes wir tun unser bestes um ihnen zu helfen herr mueller herr mueller im krankenhaus werden sie gut versorgt werden herr mueller wir werden sie jetzt ins krankenhaus bringen dort koennen die aerzte weitere untersuchungen durchfuehren und die ursache ihrer beschwerden genauer feststellen in ordnung ich verstehe ich werde ihnen jetzt einen intravenoesen zugang legen herr mueller das ist fuer den fall dass wir ihnen unterwegs fluessigkeit oder medikamente verabreichen muessen okay machen sie das julia wie sieht es mit der sauerstoffsaettigung aus sollten wir ihnen zusaetzlich sauerstoff geben durch die idee michael die sauerstoffsaettigung ist immer noch etwas niedrig herr mueller wir werden ihnen jetzt zusaetzlich sauerstoff geben das sollte ihnen helfen sich besser zu fuehlen alles klar danke julia ich denke wir sollten auch den blutdruck im auge behalten er ist immer noch eklig er ist immer noch etwas erhoeht ja das sollten wir wir werden ihren blutdruck herr mueller waehrend der fahrt weiterhin ueberwachen verstanden danke dass sie sich um mich kuemmern das ist unser job herr mueller wir sind gleich im krankenhaus halten sie durch julia wie sieht es mit den vitalwerten aus blutdruck ist immer noch etwas erhoeht einhundertfünfundvierzig komma neunzig aber stabil sauerstoffsaettigung ist gut die sauerstoffverguetung liegt bei zweiundneunzig prozent wir geben weiterhin sauerstoff gut halten wir das so bei herr mueller wie fuehlen sie sich ich bin etwas muede aber es geht das ist normal herr mueller wir sind gleich im krankenhaus und dort wird man sich gut um sie kuemmern michael ich denke wir sollten den blutdruck weiterhin endmaschig kontrollieren er ist zwar stabil aber immer noch erhoeht ja das sehe ich auch so lass uns alle fünfzehn minuten messen herr mueller wir werden ihren blutdruck weiterhin endmaschig kontrollieren um sicherzustellen dass er stabil bleibt ist das in ordnung fuer sie ja das ist in ordnung danke dass sie sich so gut um mich kuemmern das ist unser job herr mueller wir sind gleich im krankenhaus julia ich denke wir sollten herrn mueller in die klinik st josef bringen josef bringen die haben eine gute internistische abteilung und koennen dort die notwendigen untersuchungen durchfuehren ja das klingt nach einer guten entscheidung ich werde die klinik informieren und den transport vorbereiten herr mueller wir bringen sie jetzt in die klinik st josef dort haben sie eine spezialisierte abteilung die sich um ihre symptome kuemmern kann okay danke ich fuehle mich so muede das ist verstaendlich herr mueller sie sind in guten haenden und wir tun alles um ihnen zu helfen julia koennen wir herrn mueller im liegen transportieren sein zustand scheint stabil genug zu sein ja das sollte gehen ich werde die trage vorbereiten herr mueller wir werden sie jetzt auf die trage legen und ins krankenhaus bringen sie koennen dabei liegen bleiben okay danke julia ich habe herrn mueller auf die trage gelegt koennen wir ihn jetzt in den rettungswagen bringen herr michael ich habe bereits alles im rettungswagen vorbereitet lass uns herrn mueller jetzt transportieren herr mueller wir werden sie jetzt in den rettungswagen bringen sie liegen auf einer trage und wir werden sie vorsichtig transportieren ist das in ordnung fuer sie ja das ist in ordnung ich fuehle mich nur so muede das ist normal herr mueller sie sind krank und ihr koerper braucht ruhe wir bringen sie jetzt in die klinik wo sie die noetige behandlung bekommen julia ich werde herrn mueller in den rettungswagen schieben bitte ueberwache seine vitalzeichen waehrend des transports verstanden michael ich habe das ueberwachungsgeraet bereits angeschlossen und werde seine vitalzeichen kontinuierlich ueberwachen herr mueller wir sind jetzt im rettungswagen julia wird ihre vitalzeichen ueberwachen um sicherzustellen dass es ihnen gut geht danke ich fuehle mich sicherer wenn ich weiss dass sie mich ueberwachen das ist unser job herr mueller wir sind hier um sicherzustellen dass es ihnen gut geht wir fahren jetzt los zur klinik sd tissif julia wie sehen die vitalwerte von herrn mueller aus die vitalwerte sind stabil michael der blutdruck ist bei einhundertzwanzig komma achtzig mmhg die herzfrequenz liegt bei achtzig schlaegen pro minute und die sauerstoffsaettigung ist bei achtundneunzig es gibt keine anzeichen von atemnot oder schmerzen das klingt gut julia herr mueller wie fuehlen sie sich haben sie schmerzen oder unwohlsein nein ich fuehle mich eigentlich ganz gut nur ein bisschen muede das ist normal herr mueller ihre muedigkeit ist aufgrund ihrer krankheit wir sind gleich in der klinik sd josef und dort werden sie die benoetigte behandlung erhalten julia bitte behalte die vitalwerte im auge und informiere mich wenn es veraenderungen gibt natuerlich michael ich behalte die ueberwachungsgeraete im blick herr mueller wir sind gleich da sie sind in guten haenden wir werden sie gut versorgen julia wie sieht es aus sind die vitalwerte von herrn mueller immer noch stabil ja michael der blutdruck ist immer noch bei einhundertzwanzig komma achtzig mmhg die herzfrequenz bei achtzig schlaegen pro minute und die sauerstoffsaettigung bei achtundneunzig keine veraenderungen sehr gut julia herr mueller wie fuehlen sie sich haben sie schmerzen oder atemnot nein keine schmerzen oder atemnot ich fuehle mich nur sehr muede das ist normal herr mueller sie sind krank und ihr koerper braucht ruhe wir sind bald in der klinik sd josef und dort werden sie die benoetigte behandlung erhalten michael ich werde weiterhin die vitalwerte im auge behalten und sie ueber veraenderungen informieren danke julia das ist sehr wichtig herr mueller wir sind fuer sie da wenn sie etwas brauchen sagen sie uns bitte bescheid danke ich werde das tun leitstelle hier rettungswagen drei wir haben einen fünfundvierzigjaehrigen maennlichen patienten mit verdacht auf maehlenarmen vitalwerte sind stabil ankunft in klinik sd josef in etwa fünfzehn minuten bitte bereiten sie sich auf diagnostische verfahren vor verstanden rettungswagen drei wir informieren die klinik sd josef gute nachricht danke leitstelle ende herr mueller wir sind gleich in der klinik dort wird man sie weiter untersuchen ok danke guten tag wir bringen herrn mueller fünfundvierzig jahre alt mit verdacht auf maehlenarmen er ist ansprechbar und klagt ueber muedigkeit hat aber keine schmerzen oder atemnot die vitalwerte sind stabil vielen dank fuer die information julia das war ein guter einsatz du hast die vitalwerte gut im blick gehabt dirk michael ich denke wir haben alles richtig gemacht hoffentlich geht es herrn mueller bald besser"

In [43]:
from difflib import SequenceMatcher
import pandas as pd


# Preprocessing & Tokenisierung
ref_tokens = ref_text.lower().replace(",", "").replace(".", "").split()
hyp_tokens = hyp_text.lower().replace(",", "").replace(".", "").split()

# Alignment und OpCodes
matcher = SequenceMatcher(None, ref_tokens, hyp_tokens)
opcodes = matcher.get_opcodes()

# Extraktion der Wörter je Operation
subs_list = []  # List of (ref_word, hyp_word)
dels_list = []  # List of ref_word
ins_list = []   # List of hyp_word

for tag, i0, i1, j0, j1 in opcodes:
    if tag == "replace":
        for r, h in zip(ref_tokens[i0:i1], hyp_tokens[j0:j1]):
            subs_list.append((r, h))
    elif tag == "delete":
        for r in ref_tokens[i0:i1]:
            dels_list.append(r)
    elif tag == "insert":
        for h in hyp_tokens[j0:j1]:
            ins_list.append(h)

# Zählen der Operationen
subs_count = len(subs_list)
del_count = len(dels_list)
ins_count = len(ins_list)
N = len(ref_tokens)
wer_score = (subs_count + del_count + ins_count) / N if N > 0 else 0

# Ausgabe der Metriken
print(f"WER-Score: {wer_score:.2%}")
print(f"Anzahl Substitutionen: {subs_count}")
print(f"Anzahl Deletionen: {del_count}")
print(f"Anzahl Insertions: {ins_count}")

# Speichern der Token-Operationen in DataFrame
records = []
for r, h in subs_list:
    records.append({"operation": "substitution", "ref_token": r, "hyp_token": h})
for r in dels_list:
    records.append({"operation": "deletion", "token": r})
for h in ins_list:
    records.append({"operation": "insertion", "token": h})

df_tokens = pd.DataFrame(records)
# df_tokens ist gespeichert, wird nicht ausgegeben



WER-Score: 3.30%
Anzahl Substitutionen: 28
Anzahl Deletionen: 4
Anzahl Insertions: 21


In [40]:
ins_list

['arme',
 'sie',
 'haben',
 'ein',
 'atem',
 'fuer',
 'sie',
 'ja',
 'das',
 'ist',
 'ja',
 'das',
 'ist',
 'in',
 'ordnung',
 'prozent',
 'jan',
 'wir',
 'werden',
 'ihnen',
 'jetzt',
 'akzenterin',
 'dreihundertfünfundzwanzig',
 'mgthp',
 'geben',
 'um',
 'ihre',
 'schmerzen',
 'zu',
 'lindern',
 'ist',
 'das',
 'in',
 'ordnung',
 'fuer',
 'sie',
 'liegt',
 'ja',
 'auf',
 'vorbereitung',
 'wir',
 'haben',
 'eine',
 'er']

In [44]:
from difflib import SequenceMatcher

# Einfaches Beispiel
ref = "ich habe aspirin genommen"
hyp = "ich habe arpirin genommen"

# Tokenisierung
ref_tokens = ref.lower().split()
hyp_tokens = hyp.lower().split()

# Alignment und OpCodes holen
matcher = SequenceMatcher(None, ref_tokens, hyp_tokens)
opcodes = matcher.get_opcodes()

# Ausgabe der Tuples
print(opcodes)


[('equal', 0, 2, 0, 2), ('replace', 2, 3, 2, 3), ('equal', 3, 4, 3, 4)]


In [52]:
import jiwer
from jiwer.alignment import collect_error_counts

# 1) WER-Prozess laufen lassen
output = jiwer.process_words(
    "ich habe nausea und ein puls von 80 und heute morgen ein aspirin genommen",
    "ich habe nausea und ein puls von 80 90 und heute morgen ein arpirin genommen"
)

# 2) Dictionaries abrufen
subs_dict, ins_dict, del_dict = collect_error_counts(output)

# subs_dict: {(from_word, to_word): count, …}
# ins_dict: {inserted_word: count, …}
# del_dict: {deleted_word: count, …}

print("Substitutions:", subs_dict)  
# z. B. {("aspirin","arpirin"): 1}

print("Insertions:", ins_dict)        
# z. B. {"90": 1}

print("Deletions:", del_dict)        
# z. B. {}  (leer, weil nichts aus ref komplett fehlt)


Substitutions: defaultdict(<function collect_error_counts.<locals>.<lambda> at 0x121bb9d00>, {('aspirin', 'arpirin'): 1})
Insertions: defaultdict(<function collect_error_counts.<locals>.<lambda> at 0x121bba200>, {'90': 1})
Deletions: defaultdict(<function collect_error_counts.<locals>.<lambda> at 0x121bb85e0>, {})


In [56]:
from difflib import SequenceMatcher
import pandas as pd

# 1) Texte definieren
ref_text = "ich habe nausea und ein puls von 80 und heute morgen ein aspirin genommen"
hyp_text = "ich habe nausea und ein puls von acherbahn und heute morgen 80 arpirin genommen"

# 2) Vorverarbeitung & Tokenisierung
ref_tokens = ref_text.lower().replace(",", "").replace(".", "").split()
hyp_tokens = hyp_text.lower().replace(",", "").replace(".", "").split()

# 3) Alignment und Edit-OpCodes
matcher = SequenceMatcher(None, ref_tokens, hyp_tokens)
opcodes = matcher.get_opcodes()

# 4) Extraktion der Operationen und Zählung
records = []
subs_count = 0
dels_count = 0
ins_count = 0

for tag, i0, i1, j0, j1 in opcodes:
    if tag == "replace":
        for idx, (r, h) in enumerate(zip(ref_tokens[i0:i1], hyp_tokens[j0:j1])):
            subs_count += 1
            records.append({
                "operation": "substitution",
                "ref_token": r,
                "hyp_token": h,
                "ref_index": i0 + idx,
                "hyp_index": j0 + idx
            })
    elif tag == "delete":
        for idx, r in enumerate(ref_tokens[i0:i1]):
            dels_count += 1
            records.append({
                "operation": "deletion",
                "token": r,
                "ref_index": i0 + idx,
                "hyp_index": None
            })
    elif tag == "insert":
        for idx, h in enumerate(hyp_tokens[j0:j1]):
            ins_count += 1
            records.append({
                "operation": "insertion",
                "token": h,
                "ref_index": None,
                "hyp_index": j0 + idx
            })

# 5) WER-Score berechnen
N = len(ref_tokens)
wer_score = (subs_count + dels_count + ins_count) / N if N > 0 else 0

# 6) Ausgabe der Metriken
print(f"WER-Score: {wer_score:.2%}")
print(f"Anzahl Substitutionen: {subs_count}")
print(f"Anzahl Deletionen:     {dels_count}")
print(f"Anzahl Insertions:     {ins_count}")

# 7) DataFrame mit den Edit-Operationen
df_edits = pd.DataFrame(records)

# df_edits enthält:
#   - operation: 'substitution'|'deletion'|'insertion'
#   - ref_token, hyp_token (bei substitution)
#   - token (bei deletion/insertion)
#   - ref_index, hyp_index (Position im Token-Array)

# Optional: df_edits weiterverarbeiten oder exportieren




WER-Score: 21.43%
Anzahl Substitutionen: 3
Anzahl Deletionen:     0
Anzahl Insertions:     0


In [59]:
# 1) Installation (einmalig im Notebook)
!pip install deep-translator pandas

# 2) Imports
from deep_translator import GoogleTranslator
import pandas as pd

# 3) Deine englischen Medical Terms
input_terms = ["Diarrhea", "Nausea", "Weakness"]

# 4) Übersetzen
translations = []
for term in input_terms:
    de = GoogleTranslator(source='en', target='de').translate(term)
    # Du könntest hier später auch Synonyme ergänzen oder weitere Jargon-Varianten anhängen
    translations.append({"English": term, "German": de})

# 5) DataFrame erzeugen
df = pd.DataFrame(translations)

# 6) Ergebnis anzeigen
df


,English,German
0,Diarrhea,Durchfall
1,Nausea,Brechreiz
2,Weakness,Schwäche


In [100]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Deine deutschen Medizinbegriffe
terms = ["Übelkeit"]

# 1) Fach-Synonyme aus OpenThesaurus
def get_ot_synonyms(term):
    url = f"https://www.openthesaurus.de/synonyme/search?q={term}&format=application/json"
    resp = requests.get(url, timeout=5)
    data = resp.json()
    syns = set()
    for synset in data.get("synsets", []):
        for t in synset.get("terms", []):
            syns.add(t["term"])
    return list(syns)

# 2) Wiktionary-Synonyme via HTML-Scraping
def get_wiktionary_synonyms(term):
    url = f"https://de.wiktionary.org/wiki/{term}"
    resp = requests.get(url, timeout=5)
    syns = set()
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, "html.parser")
        # Suche nach dem Abschnitt mit der Überschrift "Synonyme"
        header = soup.find(lambda tag: tag.name == "span" and "Synonyme" in tag.text)
        if header:
            ul = header.find_next("ul")
            if ul:
                for li in ul.find_all("li"):
                    syns.update([w.strip() for w in li.text.split(",")])
    return list(syns)

# 3) OLS-Synonyme aus der EBI Ontology Lookup Service
def get_ols_synonyms(term, ontology="doid"):
    url = "https://www.ebi.ac.uk/ols/api/search"
    params = {
        "q": term,
        "ontology": ontology,
        "rows": 50  # mehr Treffer abfragen
    }
    resp = requests.get(url, params=params, timeout=5).json()
    syns = set()
    for doc in resp.get("response", {}).get("docs", []):
        for s in doc.get("synonyms", []):
            syns.add(s)
    return list(syns)

# Zusammensetzen
records = []
for term in terms:
    syn_ot   = get_ot_synonyms(term)
    syn_wik  = get_wiktionary_synonyms(term)
    syn_ols  = get_ols_synonyms(term)
    # alle Quellen zusammenführen, Duplikate entfernen, max. 10
    all_syns = list(dict.fromkeys(syn_ot + syn_wik + syn_ols))[:20]
    records.append({
        "term": term,
        "synonyms": all_syns
    })

df = pd.DataFrame(records)
df



,term,synonyms
0,Übelkeit,"[Brechreiz, Übelkeit, Bewegungskrankheit, Seek..."


In [105]:
for _, row in df.iterrows():
    print(f"Term: {row['term']}")
    for syn in row['synonyms']:
        print(f"  - {syn}")


Term: Übelkeit
  - Brechreiz
  - Übelkeit
  - Bewegungskrankheit
  - Seekrankheit
  - Schlechtsein
  - Kinetose
  - Nausea


In [106]:
syn_ot

['Brechreiz',
 'Übelkeit',
 'Bewegungskrankheit',
 'Seekrankheit',
 'Schlechtsein',
 'Kinetose',
 'Nausea']

In [107]:
syn_wikia

[]

In [108]:
syn_wiki

[]

In [109]:
syn_ols

[]

In [104]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Deine deutschen Medizinbegriffe
terms = ["Übelkeit"]

# 1) Fach-Synonyme aus OpenThesaurus
def get_ot_synonyms(term):
    url = f"https://www.openthesaurus.de/synonyme/search?q={term}&format=application/json"
    resp = requests.get(url, timeout=5)
    data = resp.json()
    syns = set()
    for synset in data.get("synsets", []):
        for t in synset.get("terms", []):
            syns.add(t["term"])
    return list(syns)

# 2) Wiktionary-Synonyme via HTML-Scraping
def get_wiktionary_synonyms(term):
    url = f"https://de.wiktionary.org/wiki/{term}"
    resp = requests.get(url, timeout=5)
    syns = set()
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, "html.parser")
        # Finde das Synonyme-Abschnitt-Überschriften-Element
        span = soup.find('span', id='Synonyme')
        if span:
            # Das UL direkt nach der Überschrift suchen
            header = span.find_parent(['h2', 'h3', 'h4'])
            if header:
                ul = header.find_next_sibling('ul')
                if ul:
                    for li in ul.find_all('li'):
                        for w in li.text.split(','):
                            syns.add(w.strip())
    return list(syns)

# 3) OLS-Synonyme aus EBI Ontology Lookup Service für HP und DOID
def get_ols_synonyms(term, ontologies=("hp", "doid")):
    syns = set()
    for ontology in ontologies:
        url = "https://www.ebi.ac.uk/ols/api/search"
        params = {"q": term, "ontology": ontology, "rows": 50}
        resp = requests.get(url, params=params, timeout=5).json()
        for doc in resp.get("response", {}).get("docs", []):
            for s in doc.get("synonyms", []):
                syns.add(s)
    return list(syns)

# Zusammensetzen
records = []
for term in terms:
    syn_ot   = get_ot_synonyms(term)
    syn_wiki = get_wiktionary_synonyms(term)
    syn_ols  = get_ols_synonyms(term)
    all_syns = list(dict.fromkeys(syn_ot + syn_wiki + syn_ols))[:10]
    records.append({
        "term": term,
        "synonyms": all_syns
    })

df = pd.DataFrame(records)
df


,term,synonyms
0,Übelkeit,"[Brechreiz, Übelkeit, Bewegungskrankheit, Seek..."


In [113]:
file_path = "/Users/mosimacnew/Downloads/2025AA/META/MRCONSO.RRF"

In [130]:
import pandas as pd

# Passe hier deinen tatsächlichen Pfad an
file_path = "/Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/MRCONSO.RRF"


# Preview first 10 lines, splitting nur in 15 Spalten + Rest
cols = ["CUI","LAT","TS","LUI","STT","SUI","ISPREF","AUI","SAUI","SCUI","SDUI","SAB","TTY","CODE","STR","SRL"]
preview = []

with open(file_path, "r", encoding="latin-1", errors="replace") as f:
    for _ in range(10):
        line = f.readline().rstrip("\n")
        # Split nur maximal in 16 Teile, damit STR korrekt alle "|" danach enthält
        parts = line.split("|", maxsplit=15)
        # Falls weniger Teile, mit leeren Strings auffüllen
        parts += [""] * (len(cols) - len(parts))
        preview.append(parts[:len(cols)])

df_preview = pd.DataFrame(preview, columns=cols)
df_preview



,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0|N|256|
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0|N|256|
2,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,,M0019694,D012711,MSHFRE,PEP,D012711,MacroagrÃ©gats d'albumine marquÃ©e Ã l'iode 131,3|N||
3,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3|N||
4,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,,M0019694,D012711,MSHFRE,ET,D012711,MacroagrÃ©gats d'albumine humaine marquÃ©e Ã ...,3|N||
5,C0000005,SWE,P,L17197376,PF,S20758072,Y,A33245954,,M0019694,D012711,MSHSWE,ET,D012711,(131)I-Makroaggregerat albumin,3|N||
6,C0000039,CZE,P,L6742182,PF,S7862052,Y,A13042554,,M0023172,D015060,MSHCZE,MH,D015060,"1,2-dipalmitoylfosfatidylcholin",3|N||
7,C0000039,CZE,S,L16793135,PF,S20338296,Y,A32647442,,M0023172,D015060,MSHCZE,ET,D015060,DPPC,3|N||
8,C0000039,CZE,S,L16793722,PF,S20342801,Y,A32637908,,M0023172,D015060,MSHCZE,ET,D015060,dipalmitoylfosfatidylcholin,3|N||
9,C0000039,CZE,S,L16794994,PF,S20342800,Y,A32637907,,M0023172,D015060,MSHCZE,ET,D015060,dipalmitoyl-fosfatidylcholin,3|N||


In [133]:
import pandas as pd

# 1) Pfad zur MRCONSO.RRF (im selben Verzeichnis wie dein Notebook)
file_path = "MRCONSO.RRF"

# 2) Spaltennamen-Liste laut UMLS-Spezifikation
cols = ["CUI","LAT","TS","LUI","STT","SUI","ISPREF","AUI","SAUI","SCUI","SDUI","SAB","TTY","CODE","STR","SRL"]

# 3) Wir lesen nur die Spalten CUI (0), LAT (1) und STR (14)
usecols = [0, 1, 14]
names = [cols[i] for i in usecols]

# 4) In Chunks einlesen, auf ENG/DEU filtern und sammeln
dfs = []
for chunk in pd.read_csv(
        file_path,
        sep="|",
        header=None,
        usecols=usecols,
        names=names,
        dtype=str,
        chunksize=200000,
        encoding="latin-1",
        on_bad_lines="skip"
    ):
    dfs.append(chunk[chunk["LAT"].isin(["ENG", "DEU"])])

# 5) Zusammenführen aller gefilterten Chunks
df_filtered = pd.concat(dfs, ignore_index=True)

# 6) Vorschau: erste 10 Zeilen
print(df_filtered.head(10))



        CUI  LAT                                       STR
0  C0000005  ENG            (131)I-Macroaggregated Albumin
1  C0000005  ENG                                (131)I-MAA
2  C0000039  ENG        1,2-dipalmitoylphosphatidylcholine
3  C0000039  ENG        1,2-dipalmitoylphosphatidylcholine
4  C0000039  ENG        1,2-Dipalmitoylphosphatidylcholine
5  C0000039  ENG        1,2 Dipalmitoylphosphatidylcholine
6  C0000039  ENG  1,2-Dihexadecyl-sn-Glycerophosphocholine
7  C0000039  ENG  1,2 Dihexadecyl sn Glycerophosphocholine
8  C0000039  ENG     1,2-Dipalmitoyl-Glycerophosphocholine
9  C0000039  ENG     1,2 Dipalmitoyl Glycerophosphocholine


In [136]:
# Nach dem Zusammenführen von df_filtered
print("Form von df_filtered:", df_filtered.shape)
# Optional:
print("Anzahl Zeilen:", df_filtered.shape[0])
print("Anzahl Spalten:", df_filtered.shape[1])


Form von df_filtered: (10490645, 3)
Anzahl Zeilen: 10490645
Anzahl Spalten: 3


In [137]:
import bisect

# 1) Liste aufbauen
entries = []
with open("MRCONSO.RRF", encoding="latin-1", errors="ignore") as f:
    for line in f:
        parts = line.split("|")
        if parts[1] in ("ENG","DEU"):
            entries.append((parts[14].lower(), parts[0]))
entries.sort(key=lambda x: x[0])

# 2) Abfrage mit bisect
def find_prefix(prefix):
    lo = bisect.bisect_left(entries, (prefix, ""))
    hi = bisect.bisect_right(entries, (prefix+"\uffff",""))
    return entries[lo:hi]

matches = find_prefix("nause")


In [138]:
matches

[('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),
 ('nausea', 'C0027497'),


In [166]:
from pymongo import MongoClient

# 1) Verbindung zu MongoDB auf Port 27018
client = MongoClient("mongodb://localhost:27018/")
db = client["umls"]

# 2) Collections
eng_col = db["mrconso-eng"]
ger_col = db["mrconso-ger"]

# 3) Englischen Term definieren
eng_term = "Neoplasm of abdomen"

# 4) CUI(s) für den englischen Term holen (exact, case-sensitive)
cuis = eng_col.distinct("CUI", {"LAT": "ENG", "STR": eng_term})
print(f"CUI(s) für '{eng_term}': {cuis}")

# 5) Alle deutschen Synonyme für diese CUI(s) abfragen
ger_synonyms = ger_col.distinct("STR", {"LAT": "GER", "CUI": {"$in": cuis}})

# 6) Ausgabe der deutschen Synonyme
print(f"\nDeutsche Synonyme für '{eng_term}':")
for syn in sorted(ger_synonyms):
    print(" -", syn)


CUI(s) für 'Neoplasm of abdomen': ['C0000735']

Deutsche Synonyme für 'Neoplasm of abdomen':
 - Abdominaltumor
 - Abdominaltumoren
 - Abdominelle Neubildung NNB
 - Abdominelles Neoplasma
 - Neoplasma, abdominelles
 - Neubildung des Abdomens
